In [1]:
import copy
import numpy as np

In [3]:
data = open('kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print ('data has %d chars, %d unique' % (data_size, vocab_size))

data has 137628 chars, 80 unique


In [4]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'A': 0, '9': 1, 'q': 2, '@': 3, 'V': 4, 'S': 5, '$': 6, 't': 7, 'v': 8, 'O': 9, '/': 10, 'i': 11, 'z': 12, 'I': 13, ';': 14, 'E': 15, 'y': 16, '(': 17, 'u': 18, '1': 19, 'P': 20, 'n': 21, '!': 22, '%': 23, 'H': 24, 'F': 25, 'x': 26, 'B': 27, '6': 28, '*': 29, 's': 30, ' ': 31, 'p': 32, '3': 33, 'l': 34, 'd': 35, '"': 36, ',': 37, 'k': 38, 'N': 39, 'f': 40, "'": 41, '0': 42, 'a': 43, 'W': 44, 'M': 45, 'c': 46, 'j': 47, 'C': 48, 'e': 49, '?': 50, 'r': 51, 'h': 52, '2': 53, 'T': 54, 'X': 55, 'K': 56, 'L': 57, '7': 58, '8': 59, ':': 60, '-': 61, 'Q': 62, '.': 63, 'D': 64, 'U': 65, 'o': 66, 'b': 67, 'R': 68, '5': 69, '\n': 70, 'Y': 71, '4': 72, 'G': 73, 'w': 74, 'J': 75, ')': 76, 'g': 77, 'ç': 78, 'm': 79}
{0: 'A', 1: '9', 2: 'q', 3: '@', 4: 'V', 5: 'S', 6: '$', 7: 't', 8: 'v', 9: 'O', 10: '/', 11: 'i', 12: 'z', 13: 'I', 14: ';', 15: 'E', 16: 'y', 17: '(', 18: 'u', 19: '1', 20: 'P', 21: 'n', 22: '!', 23: '%', 24: 'H', 25: 'F', 26: 'x', 27: 'B', 28: '6', 29: '*', 30: 's', 31: ' ', 32: 'p', 

In [6]:
vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
#hyperparameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [8]:
#model parameters

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

NameError: name 'xs' is not defined

In [21]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
    

In [14]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 u.X6u7;)n buP/SuJCpk mhB%kHyaDIBOyTbIi!8RvCECOfeG.9
etep(çe%"n-ApS8YrH"JkPç1meASGhp!$2P.!sTyWdr
Og)wNrShI%?jwaDm/rS YgkvbR$3*vJ'56medR)-fHqvr!AITfd8:b60zrof)(j7ulO?zcRJdtvç@Wj*ULu:e0ç!PdV19T@?7W:jE 3q 
----


In [16]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [9, 21, 49, 31, 79, 66, 51, 21, 11, 21, 77, 37, 31, 74, 52, 49, 21, 31, 73, 51, 49, 77, 66, 51, 31]
targets [21, 49, 31, 79, 66, 51, 21, 11, 21, 77, 37, 31, 74, 52, 49, 21, 31, 73, 51, 49, 77, 66, 51, 31, 5]


In [22]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter    

iter 0, loss: 109.550659
----
 a**2qse?FRgç42SfTL2mOc1/NAPrT4WWQ1g;!4RdkXNuA4K 10!5z(MbWU@WFkTAI*9A,F-? *T2E/mDIlxExoBlo61Nl77OEIxehjB-4)cyX7isQH',Ok(nkxtgs5H42EhLTzC@rRFia@EkçjGVJX* H@@go!ut9M;e.9k"!"
OCjYtMmA tu:VfdFMO?6pe6i moRq 
----
iter 1000, loss: 84.410377
----
  mestsen, fohe tor iinds Morvankh soot Gndcion the to the watwon, hom comuS mo torfo fouthe the acou, vor whk s he therundl, hithe pod onay ousamrain, bit or, the hag bead and olg oskuthre nl; he ksti 
----
iter 2000, loss: 67.223711
----
 ule seere be.er, freoms hee comkee -ur nhe weeak joite
ukest wo chey aleade w.terhe het work thag. the cohe sealy. acyaavee wholl mithegin dorevd coselk. seof fowefame reowasrss?st. ather and his hith 
----
iter 3000, loss: 58.417124
----
  to aplede pledy wthang whalk thavenish slom ther nong nton;; shar mingoy cupeby who hid wlenoeg fonguscther brar sab. afubesto hy of ither; wheast bean hiswe faven, ceningo conthend wintsighe basstoo 
----
iter 4000, loss: 53.893584
----
  llegad A

iter 35000, loss: 43.957714
----
 tos, it beer but had the Preglatce. In on the fussinges. But thet to the foor had his hape afthis worcter the do age there, doun at sid to he was thand sough his furse placper, no howtoon to the fiini 
----
iter 36000, loss: 42.588259
----
 ed ecee for back wasfeime at hin. "Way wert padion's bofing her no had evaufingitage." ho keal had the withreand, jown Gored at begon yot that it vitely viquged father. He wasted then hat sad is quo h 
----
iter 37000, loss: 42.449747
----
 outer her him pent a ont awably hay Gregor to would on wither. He romen dims oking well alacs was wot it a how with a stan, doubor's his hazk oncerped. The wourd watwon acete, costered aplist whelr he 
----
iter 38000, loss: 44.463950
----
 oray round bo this sare then ragrwert Gregor when a pratyecke would hook, foon the d
picakif thith she betcapcrenfay, ot noor to mor ive fithorsibppr and copecapsion, "Projecterunt couls maf
dochain s 
----
iter 39000, loss: 46.463238
----
 ha

iter 69000, loss: 40.717428
----
 ires exure. As inte notryout ablived and light in not been, just in collle forg-tached on the fol. Gut his that stal
sen him his formsect. Fele pen him docould the reafing ald of had the dilate been c 
----
iter 70000, loss: 40.555682
----
 amating to shew shooy, the frearinbered to his mother eveneef the linged would orcows back as of haw plelan might as hoor; flowly wain fiedning anghing of he slibured by in and it was himpleded the to 
----
iter 71000, loss: 42.196175
----
 rnat. Som that no Prope the sulither gatenblowir to other and the it would grchushing side do
Ebone tear wan sies sto now yoy wortt hom.

NoL",
 fustert thaige he he wragh 1.E Project's fraps any, Gre 
----
iter 72000, loss: 44.165914
----
 eple the siter the emat of wher star, the de
to beo he on age werl the reabidulf arding but up Giles to groned the light it hus, liglald the kentend noway, (ouring your, nticlition. The avier, plive t 
----
iter 73000, loss: 42.446045
----
 d 